<a href="https://colab.research.google.com/github/vblagoje/notebooks/blob/main/haystack2x-experiments/chat_hf_local_summarize_graham.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y haystack-ai llmx transformers

Found existing installation: haystack-ai 2.0.0b4
Uninstalling haystack-ai-2.0.0b4:
  Successfully uninstalled haystack-ai-2.0.0b4
Found existing installation: transformers 4.37.0.dev0
Uninstalling transformers-4.37.0.dev0:
  Successfully uninstalled transformers-4.37.0.dev0


In [2]:
!pip install -q autoawq accelerate git+https://github.com/huggingface/transformers.git git+https://github.com/deepset-ai/haystack.git@hf_chat_support

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch

from haystack.components.generators.utils import default_streaming_callback
from haystack.components.builders import DynamicChatPromptBuilder
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.generators.chat import HuggingFaceLocalChatGenerator
from haystack.dataclasses import ChatMessage
from haystack import Pipeline

In [4]:
lcf = LinkContentFetcher(user_agents=["Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36"])
html_converter = HTMLToDocument(extractor_type="ArticleExtractor")

prompt_builder = DynamicChatPromptBuilder(runtime_variables=["documents"])

In [5]:
llm = HuggingFaceLocalChatGenerator(model="TheBloke/OpenHermes-2.5-Mistral-7B-16k-AWQ",
                                    huggingface_pipeline_kwargs={"device_map": "auto"},
                                    streaming_callback=default_streaming_callback)

In [6]:
pipe = Pipeline()
pipe.add_component("fetcher", lcf)
pipe.add_component("converter", html_converter)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)


pipe.connect("fetcher.streams", "converter.sources")
pipe.connect("converter.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.messages")

In [7]:
template_prefix = """Given the article below: \n
            {% for document in documents %}
                {{ document.content }}
            {% endfor %}
            {{prompt_suffix}}
            """

messages = [ChatMessage.from_user(template_prefix)]


In [9]:
result = pipe.run(data={"urls": ["https://www.paulgraham.com/superlinear.html"],
                        "prompt_source": messages,
                        "template_variables": {"prompt_suffix" : "Summarize the main takeaways and learnings"},
                        "generation_kwargs": {"prompt_lookup_num_tokens": 10}})

The article discusses the concept of superlinear returns, which refers to the idea that the returns for performance are not proportional to the input, but rather grow exponentially. The author explains that this phenomenon is prevalent in various aspects of life, such as business, fame, power, military victories, knowledge, and even the benefit to humanity. The two main causes of superlinear returns are exponential growth and thresholds. Exponential growth leads to thresholds, which in turn lead to superlinear returns. The author suggests that ambitious individuals should seek work that compounds, meaning work that grows exponentially or teaches you something, and emphasizes the importance of curiosity, interest, and independence in finding fields with superlinear returns. The article also highlights the increasing variation in outcomes due to the shift away from institutions and the importance of learning from failure. The author provides a recipe for achieving superlinear returns, wh